### 어린이집 시각화

In [3]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, mapping
import folium

# === 1. 광진구 경계 불러오기 ===
shp_path = "Data/3. 2024년 2분기 기준 행정동 경계/bnd_dong_00_2024_2Q.shp"
gdf_emd = gpd.read_file(shp_path)

if gdf_emd.crs is None:
    gdf_emd.set_crs(epsg=5179, inplace=True)

gwangjin_gdf = gdf_emd[gdf_emd["ADM_CD"].astype(str).str.startswith("11050")].copy()
gwangjin_gdf = gwangjin_gdf.to_crs(epsg=5179)

# === 2. 500m 격자 생성 및 클리핑 ===
minx, miny, maxx, maxy = gwangjin_gdf.total_bounds
grid_size = 500
grid_list = []

x = minx
while x < maxx:
    y = miny
    while y < maxy:
        cell = Polygon([
            (x, y),
            (x + grid_size, y),
            (x + grid_size, y + grid_size),
            (x, y + grid_size)
        ])
        grid_list.append(cell)
        y += grid_size
    x += grid_size

grid_gdf = gpd.GeoDataFrame(geometry=grid_list, crs="EPSG:5179")
grid_clipped = gpd.clip(grid_gdf, gwangjin_gdf)

# === 3. 지도 중심 계산 ===
center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
m = folium.Map(location=[center.y, center.x], zoom_start=14, tiles="cartodbpositron")

# === 4. 광진구 경계선 표시 ===
gwangjin_union = gwangjin_gdf.geometry.unary_union
gwangjin_union_wgs = gpd.GeoSeries([gwangjin_union], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]

folium.GeoJson(
    data=mapping(gwangjin_union_wgs),
    name="광진구 경계",
    style_function=lambda x: {"color": "gray", "weight": 3, "fillOpacity": 0}
).add_to(m)

# === 5. 격자 표시 (파란 테두리) ===
for poly in grid_clipped.to_crs(epsg=4326).geometry:
    folium.GeoJson(
        data=mapping(poly),
        style_function=lambda x: {
            "fillColor": "#00000000",
            "color": "blue",
            "weight": 2,
            "opacity": 0.6,
        }
    ).add_to(m)

# === 8. 공원 위치 표시 (핑크 CircleMarker) ===
df = pd.read_csv('Data/전국도시공원정보표준데이터.csv', encoding='euc-kr')
pattern = "광진구"
filtered = df[
    df["소재지도로명주소"].astype(str).str.contains(pattern, na=False) |
    df["소재지지번주소"].astype(str).str.contains(pattern, na=False)
].copy()

if "위도" in filtered.columns and "경도" in filtered.columns:
    for _, row in filtered.iterrows():
        try:
            lat = float(row["위도"])
            lon = float(row["경도"])
            name = row.get("공원명", "이름 없음")
            folium.CircleMarker(
                location=[lat, lon],
                radius=3,
                color="deepgreen",       # 테두리색
                fill=True,
                fill_color="green",      # 내부색
                fill_opacity=0.8,
                popup=f"🌳 {name}"
            ).add_to(m)
        except:
            continue
else:
    print("❗ 위도/경도 컬럼이 없습니다. 주소 기반 좌표 변환 필요.")

# === 8. 개별 행정동 경계 표시 ===
for geom in gwangjin_gdf.geometry:
    geom_wgs = gpd.GeoSeries([geom], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]
    folium.GeoJson(
        data=mapping(geom_wgs),
        style_function=lambda x: {"color": "gray", "weight": 3, "fillOpacity": 0}
    ).add_to(m)

# === 광진구 경계선 + 채우기 (하늘색) ===
folium.GeoJson(
    data=mapping(gwangjin_union_wgs),
    name="광진구 영역",
    style_function=lambda x: {
        "color": "gray",        # 경계선 색상
        "weight": 2,             # 경계선 두께
        "fillColor": "skyblue",  # 내부 채움색
        "fillOpacity": 0.3       # 채움 투명도
    }
).add_to(m)

# 결과 출력
m


/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_3524/4092624367.py:39: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_3524/4092624367.py:39: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_3524/4092624367.py:43: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  gwangjin_union = gwangjin_gdf.geometry.unary_union


## 공원 위치

In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, mapping
import folium

# === 1. 광진구 경계 불러오기 ===
shp_path = "Data/3. 2024년 2분기 기준 행정동 경계/bnd_dong_00_2024_2Q.shp"
gdf_emd = gpd.read_file(shp_path)

if gdf_emd.crs is None:
    gdf_emd.set_crs(epsg=5179, inplace=True)

gwangjin_gdf = gdf_emd[gdf_emd["ADM_CD"].astype(str).str.startswith("11050")].copy()
gwangjin_gdf = gwangjin_gdf.to_crs(epsg=5179)

# === 2. 500m 격자 생성 및 클리핑 ===
minx, miny, maxx, maxy = gwangjin_gdf.total_bounds
grid_size = 500
grid_list = []

x = minx
while x < maxx:
    y = miny
    while y < maxy:
        cell = Polygon([
            (x, y),
            (x + grid_size, y),
            (x + grid_size, y + grid_size),
            (x, y + grid_size)
        ])
        grid_list.append(cell)
        y += grid_size
    x += grid_size

grid_gdf = gpd.GeoDataFrame(geometry=grid_list, crs="EPSG:5179")
grid_clipped = gpd.clip(grid_gdf, gwangjin_gdf)

# === 3. 지도 중심 계산 ===
center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
m = folium.Map(location=[center.y, center.x], zoom_start=14, tiles="cartodbpositron")

# === 4. 광진구 경계선 표시 ===
gwangjin_union = gwangjin_gdf.geometry.unary_union
gwangjin_union_wgs = gpd.GeoSeries([gwangjin_union], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]

folium.GeoJson(
    data=mapping(gwangjin_union_wgs),
    name="광진구 경계",
    style_function=lambda x: {"color": "gray", "weight": 3, "fillOpacity": 0}
).add_to(m)

# === 5. 격자 표시 (파란 테두리) ===
for poly in grid_clipped.to_crs(epsg=4326).geometry:
    folium.GeoJson(
        data=mapping(poly),
        style_function=lambda x: {
            "fillColor": "#00000000",
            "color": "blue",
            "weight": 2,
            "opacity": 0.6,
        }
    ).add_to(m)

# === 어린이집 위치 표시 (주황 CircleMarker) ===
df_daycare = pd.read_csv("Data/광진구_실제운영중_어린이집.csv")

if "시설 위도(좌표값)" in df_daycare.columns and "시설 경도(좌표값)" in df_daycare.columns:
    for _, row in df_daycare.iterrows():
        try:
            lat = float(row["시설 위도(좌표값)"])
            lon = float(row["시설 경도(좌표값)"])
            name = row.get("어린이집명", "이름 없음")
            folium.CircleMarker(
                location=[lat, lon],
                radius=2,
                color="darkorange",        # 테두리 색
                fill=True,
                fill_color="orange",       # 내부 색
                fill_opacity=0.9,
                popup=f"🏫 {name}"
            ).add_to(m)
        except:
            continue
else:
    print("❗ '시설 위도(좌표값)', '시설 경도(좌표값)' 컬럼이 없습니다.")

# === 8. 개별 행정동 경계 표시 ===
for geom in gwangjin_gdf.geometry:
    geom_wgs = gpd.GeoSeries([geom], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]
    folium.GeoJson(
        data=mapping(geom_wgs),
        style_function=lambda x: {"color": "gray", "weight": 3, "fillOpacity": 0}
    ).add_to(m)

# === 광진구 경계선 + 채우기 (하늘색) ===
folium.GeoJson(
    data=mapping(gwangjin_union_wgs),
    name="광진구 영역",
    style_function=lambda x: {
        "color": "gray",        # 경계선 색상
        "weight": 2,             # 경계선 두께
        "fillColor": "skyblue",  # 내부 채움색
        "fillOpacity": 0.3       # 채움 투명도
    }
).add_to(m)

# 결과 출력
m

/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_3733/1432250294.py:39: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_3733/1432250294.py:39: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_3733/1432250294.py:43: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  gwangjin_union = gwangjin_gdf.geometry.unary_union


In [17]:
grid_count = grid_clipped.shape[0]
print(f"광진구 내부 200m 격자는 총 {grid_count}개입니다.")

광진구 내부 200m 격자는 총 93개입니다.


## 청크별 빌라 개수 시각화

In [72]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, Point, mapping
import folium

shp_path = "Data/3. 2024년 2분기 기준 행정동 경계/bnd_dong_00_2024_2Q.shp"
gdf_emd = gpd.read_file(shp_path)

if gdf_emd.crs is None:
    gdf_emd.set_crs(epsg=5179, inplace=True)


gwangjin_gdf = gdf_emd[gdf_emd["ADM_CD"].astype(str).str.startswith("11050")].copy()
gwangjin_gdf = gwangjin_gdf.to_crs(epsg=5179)

# === 2. 500m 격자 생성 ===
minx, miny, maxx, maxy = gwangjin_gdf.total_bounds
grid_size = 500
grid_list = []

x = minx
while x < maxx:
    y = miny
    while y < maxy:
        cell = Polygon([
            (x, y),
            (x + grid_size, y),
            (x + grid_size, y + grid_size),
            (x, y + grid_size)
        ])
        grid_list.append(cell)
        y += grid_size
    x += grid_size

grid_gdf = gpd.GeoDataFrame(geometry=grid_list, crs="EPSG:5179")
grid_clipped = gpd.clip(grid_gdf, gwangjin_gdf)

# === 3. 빌라 좌표 데이터 불러오기 ===
df_villa = pd.read_csv("Data/빌라데이타.csv")
df_villa["위도"] = pd.to_numeric(df_villa["위도"], errors="coerce")
df_villa["경도"] = pd.to_numeric(df_villa["경도"], errors="coerce")
df_villa["세대수"] = pd.to_numeric(df_villa["세대수"], errors="coerce")  # ⭐ 추가
df_villa = df_villa.dropna(subset=["위도", "경도", "세대수"])             # ⭐ 수정


# 빌라 GeoDataFrame으로 변환 (좌표계: EPSG:4326 → EPSG:5179로 변환 필요)
gdf_villa = gpd.GeoDataFrame(
    df_villa,
    geometry=gpd.points_from_xy(df_villa["경도"], df_villa["위도"]),
    crs="EPSG:4326"
).to_crs(epsg=5179)

# === 4. 각 격자별로 빌라 수 세기 ===
def sum_households(polygon):
    villas_in = gdf_villa[gdf_villa.within(polygon)]
    return villas_in["세대수"].sum()

grid_clipped["세대수합계"] = grid_clipped.geometry.apply(sum_households)  # ⭐ 변경

# === 5. 지도 중심 계산 ===
center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
m = folium.Map(location=[center.y, center.x], zoom_start=14, tiles="cartodbpositron")


# === 6. 격자와 세대수 표시 ===
for idx, row in grid_clipped.iterrows():
    geom = row["geometry"]
    total_hh = int(row["세대수합계"])  # 소수점 방지

    # EPSG:4326 변환
    geom_wgs = gpd.GeoSeries([geom], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]

    # 격자 표시
    folium.GeoJson(
        data=mapping(geom_wgs),
        style_function=lambda x: {
            "fillColor": "#00000000",
            "color": "blue",
            "weight": 2,
            "opacity": 0.7,
        }
    ).add_to(m)

    # 중심 좌표 (EPSG:4326)
    centroid = geom.centroid
    centroid_wgs = gpd.GeoSeries([centroid], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]

    # 텍스트로 총 세대수 표시
    folium.map.Marker(
        location=[centroid_wgs.y, centroid_wgs.x],
        icon=folium.DivIcon(
            icon_size=(50, 20),
            icon_anchor=(10, 10),
            html=f'<div style="font-size: 9pt; color: red;"><b>{total_hh}</b></div>'
        )
    ).add_to(m)

# === 7. 광진구 경계선 표시 ===
gwangjin_union = gwangjin_gdf.unary_union
gwangjin_union_wgs = gpd.GeoSeries([gwangjin_union], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]

folium.GeoJson(
    data=mapping(gwangjin_union_wgs),
    name="광진구 경계",
    style_function=lambda x: {"color": "black", "weight": 3, "fillOpacity": 0}
).add_to(m)

# === 8. 개별 행정동 경계 표시 ===
for geom in gwangjin_gdf.geometry:
    geom_wgs = gpd.GeoSeries([geom], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]
    folium.GeoJson(
        data=mapping(geom_wgs),
        style_function=lambda x: {"color": "gray", "weight": 5, "fillOpacity": 0}
    ).add_to(m)

# 결과 보기
m

/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_37626/1841706458.py:62: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_37626/1841706458.py:62: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_37626/1841706458.py:100: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  gwangjin_union = gwangjin_gdf.unary_union


In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# === 1. 빌라 데이터 로드 ===
villa_df = pd.read_csv("Data/빌라데이타.csv")

# === 2. 위도/경도로 포인트 만들기 ===
villa_gdf = gpd.GeoDataFrame(
    villa_df,
    geometry=gpd.points_from_xy(villa_df["경도"], villa_df["위도"]),
    crs="EPSG:4326"
)

# === 3. 행정동 경계 파일 로드 ===
dong_gdf = gpd.read_file("Data/3. 2024년 2분기 기준 행정동 경계/bnd_dong_00_2024_2Q.shp")

# === 4. 좌표계 맞추기 (WGS84로 통일) ===
if dong_gdf.crs is None:
    dong_gdf.set_crs(epsg=5179, inplace=True)

dong_gdf = dong_gdf.to_crs(epsg=4326)

# === 5. 공간조인: 빌라 ↔ 행정동 경계 ===
joined = gpd.sjoin(
    villa_gdf,
    dong_gdf[["ADM_CD", "ADM_NM", "geometry"]],
    how="left",
    predicate="within"
)

# === 6. 컬럼 정리 ===
joined = joined.rename(columns={
    "ADM_CD": "행정동코드",
    "ADM_NM": "행정동명"
})

# 필요한 컬럼만 보기 좋게 정리
result = joined[["위도", "경도", "행정동명", "행정동코드"] + [col for col in villa_df.columns if col not in ["위도", "경도"]]]

# 결과 확인
print(result.head())

# 저장하고 싶다면:
result.to_csv("빌라_행정동매핑.csv", index=False)

          위도          경도  행정동명     행정동코드                   대지위치     시군구코드명  \
0  37.537760  127.064704  자양4동  11050670    서울특별시 광진구 자양동 843-1  서울특별시 광진구   
1  37.535979  127.075837  자양3동  11050660  서울특별시 광진구 자양동 227-273  서울특별시 광진구   
2  37.561104  127.079313  중곡1동  11050550   서울특별시 광진구 중곡동 257-21  서울특별시 광진구   
3  37.561599  127.081699  중곡1동  11050550   서울특별시 광진구 중곡동 163-35  서울특별시 광진구   
4  37.566586  127.085604  중곡3동  11050570    서울특별시 광진구 중곡동 29-22  서울특별시 광진구   

  법정동코드명   대지면적    건축면적 주용도코드명   세대수    구  건축면적_결측  
0    자양동  270.0  156.22   공동주택  16.0  광진구    False  
1    자양동  245.9  147.50   공동주택  10.0  광진구    False  
2    중곡동  156.7   92.57   공동주택   3.0  광진구    False  
3    중곡동    0.0   57.85   공동주택   2.0  광진구    False  
4    중곡동  223.5  133.74   공동주택   6.0  광진구    False  


In [14]:
result.columns

Index(['위도', '경도', '행정동명', '행정동코드', '대지위치', '시군구코드명', '법정동코드명', '대지면적', '건축면적',
       '주용도코드명', '세대수', '구', '건축면적_결측'],
      dtype='object')

In [17]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# === 파일 불러오기 ===
villa_df = pd.read_csv("Data/빌라_행정동매핑.csv")
population_df = pd.read_csv("Data/행정동별_인구비율_데이터.csv")
daycare_df = pd.read_csv("Data/광진구_실제운영중_어린이집.csv")
park_df = pd.read_csv("Data/전국도시공원정보표준데이터.csv", encoding='euc-kr')

# === 좌표 GeoDataFrame 변환 ===
villa_gdf = gpd.GeoDataFrame(
    villa_df,
    geometry=gpd.points_from_xy(villa_df["경도"], villa_df["위도"]),
    crs="EPSG:4326"
).to_crs(epsg=5179)

daycare_gdf = gpd.GeoDataFrame(
    daycare_df,
    geometry=gpd.points_from_xy(daycare_df["시설 경도(좌표값)"], daycare_df["시설 위도(좌표값)"]),
    crs="EPSG:4326"
).to_crs(epsg=5179)

park_df = park_df[
    park_df["소재지도로명주소"].astype(str).str.contains("광진구", na=False) |
    park_df["소재지지번주소"].astype(str).str.contains("광진구", na=False)
]
park_gdf = gpd.GeoDataFrame(
    park_df,
    geometry=gpd.points_from_xy(park_df["경도"], park_df["위도"]),
    crs="EPSG:4326"
).to_crs(epsg=5179)

# === 격자 생성 후 클리핑된 grid_clipped 사용 ===
# 이전에 생성한 grid_clipped 변수를 그대로 사용해야 합니다

# === 격자별 통계 계산 ===
results = []

for idx, grid in grid_clipped.iterrows():
    grid_geom = grid.geometry
    grid_id = idx

    # 어린이집 수
    daycare_count = daycare_gdf[daycare_gdf.intersects(grid_geom)].shape[0]

    # 공원 수
    park_count = park_gdf[park_gdf.intersects(grid_geom)].shape[0]

    # 빌라: 세대수 합산
    villas_in_grid = villa_gdf[villa_gdf.intersects(grid_geom)]
    total_villa_households = villas_in_grid["세대수"].sum()

    # 행정동 비율 계산
    admin_ratio = (
        villas_in_grid.groupby("행정동명")["세대수"].sum() / total_villa_households
        if total_villa_households > 0 else pd.Series()
    )

    # 가중 평균 인구 구성 계산
    pop_dist = {}
    for dong, ratio in admin_ratio.items():
        row = population_df[population_df["행정동"] == dong]
        if not row.empty:
            for col in row.columns[1:]:
                pop_dist[col] = pop_dist.get(col, 0) + row.iloc[0][col] * ratio

    results.append({
        "격자ID": grid_id,
        "어린이집수": daycare_count,
        "공원수": park_count,
        "빌라세대수": int(total_villa_households),
        **pop_dist
    })

# === 결과 저장 ===
grid_stats_df = pd.DataFrame(results)
grid_stats_df.to_csv("광진구_격자_요약정보.csv", index=False)